<img src="mbit_logo.png" alt="drawing" align="right" style="float" width="150" height="150"/>

<font color="#D31525"><h3 align="left">Detección de fraude en transacciones financieras</h3></font>
<font color="#2C3E50"><h3 align="left">ENSUCIAR EL DATASET</h3></font>

## Importar librerias

En esta primera parte del código, se realizan las llamadas a las librerías:

In [1]:
# Paquetes de manipulación de datos
import pandas as pd
import numpy as np
import boto3
import s3fs


# Paquetes de visualización
import matplotlib.pyplot as plt
import seaborn as sns

## Importar Dataset

Para desarrollar el trabajo, se ha elegido un dataset generado a partir del simulador **PaySim**, que genera conjuntos de datos sintéticos similares a los conjuntos de datos reales de las transacciones de dinero móvil. PaySim utiliza datos agregados para generar un conjunto de datos sintéticos que se asemeje al funcionamiento normal de las transacciones y además inyecte comportamientos maliciosos.

En este caso, la muestra extraída corresponde a 1 mes de un servicio de dinero móvil implementado en un país africano.

*PaySim first paper of the simulator:
E. A. Lopez-Rojas , A. Elmir, and S. Axelsson. "PaySim: A financial mobile money simulator for fraud detection". In: The 28th European Modeling and Simulation Symposium-EMSS, Larnaca, Cyprus. 2016*

Leemos el fichero usando la libreria **boto3**

In [2]:
s3 = boto3.client("s3")

In [39]:
# Metemos en la variable res, el nombre de todos los buckets que tenemos en nuestra máquina de AWS
res = s3.list_buckets()

In [8]:
# Mostramos los distintos buckets que tenemos en S3
res["Buckets"]

[{'Name': 'aasmbitschool',
  'CreationDate': datetime.datetime(2020, 3, 23, 18, 17, 50, tzinfo=tzlocal())},
 {'Name': 'tfmfraud',
  'CreationDate': datetime.datetime(2020, 8, 12, 10, 27, 52, tzinfo=tzlocal())}]

In [10]:
# Seleccionamos el bucket con el que vamos a trabajar
BUCKET_NAME = 'tfmfraud'

In [11]:
# Listamos los ficheros que tenemos dentro del bucket que hemos solucionado.
s3.list_objects(Bucket = BUCKET_NAME)

{'ResponseMetadata': {'RequestId': '2097D59B73B84492',
  'HostId': 'FDe1CAnoRVCrlfo2+fScVz+ADLn72F7Qr3LvQE8ktEu7bx+XSN01dsPKRTUN+FCNE7LgRMfyuLI=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'FDe1CAnoRVCrlfo2+fScVz+ADLn72F7Qr3LvQE8ktEu7bx+XSN01dsPKRTUN+FCNE7LgRMfyuLI=',
   'x-amz-request-id': '2097D59B73B84492',
   'date': 'Wed, 12 Aug 2020 15:14:49 GMT',
   'x-amz-bucket-region': 'us-east-1',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'IsTruncated': False,
 'Marker': '',
 'Contents': [{'Key': 'dataset_v0.csv',
   'LastModified': datetime.datetime(2020, 8, 12, 10, 43, 49, tzinfo=tzlocal()),
   'ETag': '"3ad3f545cf41c2d46802c39d6c6e6d3e-29"',
   'Size': 493534783,
   'StorageClass': 'STANDARD',
   'Owner': {'DisplayName': 'awslabsc0w601512t1581883282',
    'ID': 'd79b5e3eed7c637ad447a28498162e365200293b12dd2095a264060d2a7898d4'}}],
 'Name': 'tfmfraud',
 'Prefix': '',
 'MaxKeys': 1000,
 'Encoding

In [16]:
# Descargamos el fichero del bucket de s3 a la máquina EC2 para poder trabajar con él.
s3.download_file(Bucket = BUCKET_NAME, Key = 'dataset_v0.csv',Filename = '/tmp/dataset_v0.csv')

In [18]:
# Listamos los ficheros que hay en la ruta /tmp. Esta ruta es donde hemos descargado nuestro fichero de s3
!ls /tmp

dataset_v0.csv
systemd-private-6c440eefc28c4d319793e82277e9b4cc-chronyd.service-MUrd4O


In [19]:
#Leemos el fichero y lo metemos en un dataframe.
df_orig = pd.read_csv('/tmp/dataset_v0.csv', dtype={'rank':'category'})

## Análisis exploratorio de datos inicial

Tomemos sensibilidad del dataset de trabajo

In [20]:
print("Tamaño dataset del dataset:", df_orig.shape)
print("Número de variables/predictores (columnas):", df_orig.shape[1])
print("Número de registros (instancias/filas):", df_orig.shape[0])

Tamaño dataset del dataset: (6362620, 11)
Número de variables/predictores (columnas): 11
Número de registros (instancias/filas): 6362620


In [21]:
df_orig.head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.00,160296.36,M1979787155,0.0,0.00,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.00,19384.72,M2044282225,0.0,0.00,0,0
2,1,TRANSFER,181.00,C1305486145,181.00,0.00,C553264065,0.0,0.00,1,0
3,1,CASH_OUT,181.00,C840083671,181.00,0.00,C38997010,21182.0,0.00,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.00,29885.86,M1230701703,0.0,0.00,0,0
5,1,PAYMENT,7817.71,C90045638,53860.00,46042.29,M573487274,0.0,0.00,0,0
6,1,PAYMENT,7107.77,C154988899,183195.00,176087.23,M408069119,0.0,0.00,0,0
7,1,PAYMENT,7861.64,C1912850431,176087.23,168225.59,M633326333,0.0,0.00,0,0
8,1,PAYMENT,4024.36,C1265012928,2671.00,0.00,M1176932104,0.0,0.00,0,0
9,1,DEBIT,5337.77,C712410124,41720.00,36382.23,C195600860,41898.0,40348.79,0,0


Para simular el servicio de dinero móvil, se necesitan simular adecuadamente los diferentes tipos de transacciones que el sistema soporta. En el simulador **PaySim** se ha decidido cubrir 5 de los tipos de transacciones más importantes: CASH-IN, CASH-OUT, DEBIT, PAYMENT y TRANSFER. Estas categorías se recogen en la variable *type*:
* CASH-IN - Proceso de aumentar el balance de una cuenta pagando en efectivo a un comerciante
* CASH-OUT - Proceso opuesto a CASH-IN. Supone retirar dinero de la cuenta de un comerciante, disminuyendo el balance de la misma
* DEBIT - Similar a CASH-OUT. Consiste en enviar el dinero del servicio de dinero móvil a una cuenta bancaria
* PAYMENT - Proceso de pagar por bienes o servicios a los comerciantes que disminuye el saldo de la cuenta y aumenta el saldo del receptor
* TRASNFER - Proceso de enviar dinero a otro usuario del servicio a través de la plataforma de dinero móvil

Adicionalmente, el dataset cuenta con 9 variables más que determinan el movimiento financiero:
* *Amount* - Importe de la operación en moneda local
* *nameOrig* - Cliente que inició la operación
* *oldbalanceOrig* - Saldo inicial antes de la transacción en la cuenta de origen
* *newbalanceOrig* - Nuevo saldo después de la transacción en la cuenta de origen
* *nameDest* - Cliente destinatario de la operación
* *oldbalanceDest* - Saldo inicial del receptor antes de la transacción
* *newbalanceDest* - Nuevo saldo del receptor después de la transacción
* *isFraud* - Indicador de transacción realizadas por agentes fraudulentos
* *isFlaggedFraud* - Indicador de transacción ilegal. El modelo de negocio tiene como objetivo controlar las transferencias masivas de una cuenta a otra y marca los intentos ilegales. Un intento ilegal en este conjunto de datos es un intento de transferir más de 200.000 en una sola transacción.

Y una variable que indica el momento de tiempo en que se realizó el movimiento, *step*. Esta variable mapea 1 hora de tiempo real. El dataset contiene información de 31 días, por lo que en el data set encontramos 743 steps.

Una vez conocida la estructura del dataset, continuamos realizando el análisis de datos inicial. Identificamos la tipología de las variables:

In [22]:
df_orig.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

Identificamos los estadísticos básicos de las variables numéricas:

In [23]:
df_orig[['step','amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 
        'isFraud', 'isFlaggedFraud']].describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


Evaluamos si la existencia de datos missing:

In [24]:
df_orig.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Evaluamos si existen filas duplicadas:

In [25]:
df_orig.duplicated().sum()

0

Tras este primer análisis, observamos que nuestro dataset está bastante limpio. Para poder aplicar técnicas de de preprocesamiento de datos aprendidas durante el curso, vamos a ensuciar nuestro dataset. Para ello:  
1. *Creamos un nuevo dataset a partir de un subconjunto aleatorio de filas de dataset original*
2. *Introducimos el 10% de valores missing en las diferentes columnas*  
3. *Insertaremos las nuevas filas en el dataset original*

## Generación de nuevo dataset 'sucio'
Este nuevo dataset, contendrá la información original (6.362.620 filas) y un subconjunto de datos 'sucios' con los que se trabajará en la fase de preprocesado. Para ello, llevamos a cabo los siguientes pasos:

**1. Obtenemos una muestra aleatoria a partir del dataset original**  
El método más fácil para obtener una submuestra aleatoria de un objeto DataFrame es mediante el método `sample()`. El número de registro se puede indicar mediante el parámetro `n`. Escogeremos un **4% de filas del dataset original**.

In [26]:
df_sample = df_orig.sample(frac=0.04, random_state=1)
print("Tamaño dataset del dataset:", df_sample.shape)

Tamaño dataset del dataset: (254505, 11)


**2. Insertar aleatoriamente** `np.nan` **en el dataset**  
A continuación, anulamos un 10% de valores para cada columna del datafram `df_sample` (fijamos la semilla de números aleatrorios para que siempre se conviertan a missing el mismo valor)  
URL: https://www.it-swarm-es.tech/es/python/inserte-aleatoriamente-los-valores-de-na-en-pandas-dataframe/826780756/

In [28]:
np.random.seed(324)

In [29]:
fixed = df_sample.iloc[:,0:2]
not_fixed = df_sample.iloc[:,2:11]

In [30]:
df_sample_one = not_fixed.mask(np.random.random(not_fixed.shape) < .1)

In [31]:
df_concat = pd.concat([fixed, df_sample_one], axis=1, sort=False)

In [32]:
df_concat.isnull().sum()

step                  0
type                  0
amount            25469
nameOrig          25811
oldbalanceOrg     25691
newbalanceOrig    25318
nameDest          25585
oldbalanceDest    25452
newbalanceDest    25450
isFraud           25419
isFlaggedFraud    25580
dtype: int64

In [33]:
df_concat.loc[(df_concat['amount'].isnull()) | (df_concat['nameOrig'].isnull()) | (df_concat['oldbalanceOrg'].isnull())
            | (df_concat['newbalanceOrig'].isnull()) | (df_concat['nameDest'].isnull()) | (df_concat['oldbalanceDest'].isnull()) 
             | (df_concat['newbalanceDest'].isnull()) | (df_concat['isFraud'].isnull())].head(5)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
6322570,688,CASH_IN,NaN,C867750533,NaN,31616.12,C1026934669,169508.66,145951.53,0.0,0.0
3621196,274,PAYMENT,6236.13,NaN,0.00,0.00,M701283411,0.00,NaN,0.0,0.0
1226256,133,PAYMENT,33981.87,C279540931,18745.72,NaN,M577905776,0.00,0.00,0.0,0.0
3201247,249,CASH_OUT,NaN,C530649214,20765.00,0.00,C1304175579,252719.19,404732.93,0.0,0.0
5422829,378,CASH_OUT,520230.74,NaN,0.00,0.00,C1640500532,540059.79,1060290.53,0.0,0.0


**3. Ingesta de datos 'sucios' en el dataset original**  
Insertamos el nuevo conjunto de datos en el dataset original

In [35]:
frames = [df_orig, df_concat]
df_dirty = pd.concat(frames)

**4. Guardamos el nuevo dataset**  
Guardamos el fichero usando en nuestro bucket de s3 usando la libreria **boto3**

In [36]:
df_dirty.to_csv('/tmp/df_dirty.csv', index = False)

In [37]:
# chequeamos que en la ruta /tmp de la máquina se ha generado el fichero df_dirty.csv
!ls /tmp

dataset_v0.csv
df_dirty.csv
systemd-private-6c440eefc28c4d319793e82277e9b4cc-chronyd.service-MUrd4O


In [38]:
s3.upload_file(Bucket = BUCKET_NAME, Key = 'df_dirty.csv', Filename = '/tmp/df_dirty.csv')